In [28]:
import loompy
import numpy as np

In [78]:
ds = loompy.connect("../data/Aerts_Fly_AdultBrain_Filtered_57k.loom", validate=False)

cluster = []

for row in ds.ca.ClusterID:
    cluster.append(int(row[0]))
    
cluster = np.array(cluster)

ds.ca.is_kc = np.where((cluster == 8) | (cluster == 22) | (cluster == 28), True, False)

with loompy.new("../data/STAWP_Fly_AdultBrain_KC.loom") as dsout:  # Create a new, empty, loom file
    cells = ds.ca.is_kc != 0
    for (ix, selection, view) in ds.scan(items=cells, axis=1):
        dsout.add_columns(view.layers, col_attrs=view.ca, row_attrs=view.ra)

In [79]:
ds2 = loompy.connect("../data/STAWP_Fly_AdultBrain_KC.loom", validate=False)

In [80]:
ra_list = ds2.ra.keys()
ra_list.remove('Gene')

for ra in ra_list:
    del  ds2.ra[ra]


In [81]:
ls_to_remove = ['Age', 'ClusterID', 'Clusterings', 'Embedding', 'Embeddings_X',
                'Embeddings_Y', 'Gender', 'Genotype', 'RegulonsAUC', 'Replicate', 'is_kc',
                'kc_cell']

for ca in ls_to_remove:
    del  ds2.ca[ca]

In [84]:
df = ds2.pandas(row_attr="Gene")
ds.close()
ds2.close()

In [104]:
df = df.set_index("CellID").drop(['nGene', 'nUMI', 'scluster'], axis=1)
df = df.transpose()

In [112]:
df = df.reset_index()
df.to_csv("../data/STAWP_Fly_AdultBrain_KC.csv")

Help on method transpose in module pandas.core.frame:

transpose(*args, **kwargs) method of pandas.core.frame.DataFrame instance
    Transpose index and columns.
    
    Reflect the DataFrame over its main diagonal by writing rows as columns
    and vice-versa. The property :attr:`.T` is an accessor to the method
    :meth:`transpose`.
    
    Parameters
    ----------
    copy : bool, default False
        If True, the underlying data is copied. Otherwise (default), no
        copy is made if possible.
    *args, **kwargs
        Additional keywords have no effect but might be accepted for
        compatibility with numpy.
    
    Returns
    -------
    DataFrame
        The transposed DataFrame.
    
    See Also
    --------
    numpy.transpose : Permute the dimensions of a given array.
    
    Notes
    -----
    Transposing a DataFrame with mixed dtypes will result in a homogeneous
    DataFrame with the `object` dtype. In such a case, a copy of the data
    is always made.
 